# Notebook Lecture 2: Introduction to System Modeling
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook covers the basics of system modeling, with a worked through example, how to compute the discretized output of a system, and a couple interactive examples to build an intuition of the subjects.

Authors:
- Jorit Geurts; jgeurts@ethz.ch
- Shubham Gupta; shugupta@ethz.ch

## Learning Objectives

In lecture 1 we saw that this course addresses the control of dynamical systems. As such, two of the course objectives are to understand how analyze and synthesize systems to achieve some goal (consider the cruise-examples examples in [Supporting Notebook 1](https://github.com/idsc-frazzoli/CS1-Notebooks/blob/12c89d0f00aabcf2a7d53bd9e1a78dc2f6b96bf4/supporting_notebooks/SB2401.ipynb)). However, to do so, we first need to understand how such systems can be represented mathematically (i.e., modeled).
Thus, in this notebook, we introduce what is meant by a dynamical system, what do we mean by modeling it, and then discuss modelling techniques for different types of systems.  

After completing this notebook, you should be able to:

1. Explain the concept of system modeling to another student.
2. Understand dynamic control systems in terms of inputs and outputs.
3. Understand the concept of "state" to model a system.
4. Derive a model of a dynamical system by hand and describe what their states, inputs and outputs are.
5. Understand, given an input signal and initial condition, how to compute the output of a dynamical system. (more formally introduced in Lecture 4)
6. Understand the benefits and dangers of using feedback to control a dynamical system.

### Import the packages:

The following cell imports the required packages. Run it before running the rest of the notebook.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display

# Definitions


## Dynamical System

In general:<br>
- A dynamical (control) system generates one or many output signals given one or many input signals.

More formally:<br>
> A dynamical control system is an operator (i.e., an input-output map), $\sum$, that takes an input signal, $u$, and produces an output signal $y$ -- see image below.
These dynamical systems have internal states, $x$, also referred to as *memory*, that describe the relationship between the input and output signal.<br>

In general, dynamical systems are arbitrated to any number of input and output signals. However, in this course we will consider Single Input Single Output (SISO) systems.

<div style="text-align: center;">
    <img src="img/Dynamical_System.png" width="400">
</div>

### Example<br>

We have a vehicle, and this vehicle must reach a desired speed (i.e., reference) subject to some disturbance. In this case: 
- the *dynamical system* is the car.

<div style="text-align: center;">
    <img src="img/Dynamical_System_Example.png">
</div>

To be able to determine what *control signal* is required to achieve the reference output, a mathematical representation of the system is required.  
Thus, the rest of the notebook is dedicated to introducing techniques to model dynamical systems.  

## Modeling

>(System) Modeling for control systems is the act of formally representing the **relationship between the input(s) and output(s)** of a system.<br>
Usually this means that we want to write down the **Differential Equations (PDE/ODE)** that govern the underlying system we want to control -- in the previous example this would be the car.

Having a model of the system allows us to analyze the system, synthesis and implement in practical applications a control structure.

As mentioned in the lecture, *"All Models are wrong, but some are useful"*. This is due to the fact that in general it is not possible to exactly model all parts of a system (consider the heating system in your house, losses are present through the window, the window sills, the walls, etc...).
Thus, the underlying model will always be an approximation of reality and one needs to keep this in mind while creating and utilizing a model.

# Modeling Approaches

Let us now go through the steps of modeling a system.

### General Steps

At a high-level, the general steps to model a dynamical system are as follows:
1. Identify the relevant input, output and state variables. 
2. Derive the differential equation/s governing the dynamical system. 
3. Write the differential equations into state-space/standard form. 

We will now introduce each of the above steps in more detail. 

#### 1. Identifying System Boundaries: Inputs and Outputs

First we define some relevant terminology:
1. **Inputs:** The inputs to a system define how the outside world affects the system itself. This includes both, factors from the general environment and that of the user (who may have the goal of controlling the system for a specific task).   
For example while controlling the temperature of water in a faucet, it is affected by both the user controlling the knob and the heat dissipation in the plumbing.   
   
   Inputs can be separated into:
   * **Endogenous inputs**: Inputs that can be controlled. These are in general the control inputs (for example the faucet knob or the gas throttle in a car).
   * **Exogenous inputs**: Inputs that the user has no control over. These are what we will refer to as **disturbances** (for example wind turbulence when flying a helicopter is a disturbance).

1. **Outputs:** The outputs of a system are the measurements or observations we make of the system. 
   These can be separated in:
   * **Measured outputs**, quantities we can measure directly (e.g., the speed of a car).
   * **Performance outputs**, which we can't measure directly but would also like to control (e.g., the average fuel efficiency of a car).

2. **System State/memory:**
The **memory** of a system, summarizes the effects of all past inputs. We will refer to this as the (internal) **state** of the system. It is a vector describing the system. 
   For example, one could describe the state, $x(t)$, of a car by its position, $p(t)$ and velocity, $\dot p(t)$:    
   $$x(t) = [p(t), \dot p(t)]^T$$

1. **Parameters:**
These are attributes/features that are specific to the problem, and will not change over time. For example, this could be the geometry of the car -- but varying across different cars, the values of these parameters may vary. 

Some of the above aspects can be controlled while deriving a model, meaning we can choose what quantities will be captured in the model. Thus, to some extent we can choose the inputs to our system and the corresponding outputs to better suit our goal -- this notion will become clearer as we go through the examples in the lecture, this notebook, recitals and the problem sheet.


#### Example 

Consider the system below -- where a car is travelling along a slope, with some relevant forces depicted in the figure:

<div style="text-align: center;">
    <img src="img/car_system_example.png" width="400">
</div>


In this model we included the drag force, roll resistance, thrust (engine model) and gravity as inputs to the system.   
Another choice for inputs would be to ignore the drag force and only look at the roll resistance, thrust and gravity.   
On the other hand one could make the model more complex and explicitly model the trust force by including brake losses, or a delay between applying the gas pedal and the vehicle accelerating.
This would increase the scope of the system as new quantities such as temperatures and pressures inside the motor become relevant.
Whether the selected choice is appropriate depends on the overarching goal -- but note that a certain selection may be better than others.

Lastly we can also change the outputs of the system. We are not limited to measuring the velocity of the car, but we could instead measure acceleration, or include a GPS device and measure the position explicitly. Again, these are objective dependent considerations that vary according to the context of the problem.

Since the principal objective of the course is to teach how to control said systems, it will in general be clear by the context what the relevant inputs, outputs and system states are -- See the corresponding problem set for further examples. 

#### 2. Deriving the Governing Differential Equations

Once the system inputs and outputs are defined the governing differential equations of the system need to be derived. This enables for a formal representation of the system to be obtained, and used directly in the control architecture. 

>There are a variety of approaches towards doing this, and you will go through them in more detail in **Thermodynamics I** and **Mechanics III**. However, here we briefly introduce an overarching technique for any arbitrary system.

Firstly, a system will often comprise a so-called **storage** quantity (e.g., energy, temperature, mass, momentum, etc...). In the case of a heating system of a house we could consider temperature of the house/room. Note that it is sometimes possible to consider multiple storage quantities for a given scenario, nevertheless sometimes it is easier to pick one option over another. Deciding which storage quantity is appropriate for a given context will mostly come through practice and experience. Nevertheless, we will limit the possible types to those mentioned below.

Secondly, the **storage** quantity will typically vary, induced by the flow of some other quantities. Thus, we can formulate a differential equation as follows:
$$\frac{d}{dt}\text{storage} = \sum \text{inflows} - \sum \text{outflows}$$

Most physical laws can be interpreted or rewritten as a variation of the storage approach. Consider the heating system for a house, the rate of change of heat transfer is equal to the rate of change of heat inflows (e.g., radiator) subtracted by the rate of change of heat outflows (e.g., losses). 

The following are some of the common formulas used (with $\dot \square$  indicating the time derivative).

**Newton's 2nd Law for Linear Translation:**
$$
\sum F(t) = m a(t) = m \dot v(t) = m \ddot x(t)
$$
with $\sum F(t)$ being the sum of all forces applied to the object, $m$ being the mass of the object, $a(t)$ the acceleration, $v(t)$ the velocity and $x(t)$ the position of the object.

**Newton's 2nd Law for Rotation:**
$$
\sum T(t) = J \ddot \theta(t) = J \dot \omega (t)
$$
with $\sum T(t)$ being the sum of all torques, $J$ being the Moment of Inertia, $\omega(t)$ being the rotational velocity, and $\theta(t)$ being the rotation angle.

**Heat Transfer (Fourier's Law):**

A heat flow from surface i too surface j of a wall is proportional to the difference of temperatures: 
$$
\dot Q_{ij}(t) = c_{ij} (T_i(t)-T_j(t))
$$
where $c_{ij}$ is the thermal conductance of the wall between the two surfaces.<br>
Having a heat flow into a room from different surfaces we have the following temperature equation:
$$
C \dot T(t) = \sum \dot Q(t)
$$
with $C$ being the thermal capacity of that room.

**Inductance:** *Not exam relevant*
$$
U(t) = L \dot I(t)
$$
with $U(t)$ being the voltage, $L$ the inductance and $I(t)$ the current.

**Capacitance:** *Not exam relevant*
$$
I(t) = C \dot U(t)
$$
with $U(t)$ being the voltage, $C$ the capacitance and $I(t)$ the current.

#### 3. Formulating the Different Equations in Standard/State-Space Form

Every control system has different quantities to keep track of, different inputs, outputs, state and most importantly different orders of derivatives. Thus, we wish to transform any $n^{th}$ order ODE, into $n$ $1^{st}$ order ODEs. This will enable for unified theories to be applied to arbitrary control systems. Standard form, also referred to as *state-space* form, looks like:
$$
\dot x (t) = f(x(t), u(t))\\
y(t) = h(x(t), u(t))
$$

It is important to note that $x(t)$, $u(t)$ and $y(t)$ are vectors and on the left-hand side we only have the first derivative, i.e.:
$$
\begin{bmatrix}
    \dot x_1(t)
    \\
    \vdots
    \\
    \dot x_N(t)
\end{bmatrix}
=
\begin{bmatrix}
    f_1(x(t), u(t))
    \\
    \vdots
    \\
    f_N(x(t), u(t))
\end{bmatrix}
$$
$$
\begin{bmatrix}
    y_1(t)
    \\
    \vdots
    \\
    y_N(t)
\end{bmatrix}
=
\begin{bmatrix}
    h_1(x(t), u(t))
    \\
    \vdots
    \\
    h_N(x(t), u(t))
\end{bmatrix}\\
$$
This ensures that all systems have the same form and results are easily transferable.
In this formulation $f(x(t), u(t))$ is called the *dynamical model* and $h(x(t), u(t))$ is called the *measurement* model.  

The *standard form* of the system is commonly referred to as the **State space form** of the system, since it tracks the system evolution in the state vector space (and not just the output space)*.

Usually systems, when derived as above, are not in standard form and include higher order derivatives. To convert these higher order systems to standard form use a trick from Linear Algebra that every higher derivative can be reformulated as a system of first order derivatives.

Lets take this example differential equation here
 $$\frac{d^2}{dt^2} \theta + \frac{g}{l}sin(\theta) = 0$$  
 $$ y = \theta $$

At a first glance it is not in the standard form, but looks similar. The main problem is the derivative. So how do we resolve that?   

*Introduce an extra state*, such that the state is $$ x = [\theta, \dot \theta]^T$$ Let $x_1 = \theta, x_2 = \dot \theta$, then the system can be written as:
$$ 
\begin{bmatrix}
    \dot x_1 \\ 
    \dot x_2 
\end{bmatrix}
= 
\begin{bmatrix}
    \dot \theta \\
    \ddot \theta
\end{bmatrix}
=
\begin{bmatrix}
    x_2 \\
    -\frac{g}{l} \sin(x_1)
\end{bmatrix} 
$$

$$ y = x_1 $$

The system is now in state-space form. 

## Example System

Let us now go through these three steps using a simple spring damper system, with the goal of measuring the force exerted by the mass onto the spring -- further examples that reinforce these concepts will be introduced in the recitals and problem sheets. <br>

*Note: To motivate the purpose of such a spring damper -- This kind of spring damper systems are important for measurement systems, eigenfrequency calculation of mechanical systems, modeling material properties or calculating the damping properties of systems (for example an earthquake damper in a building).*<br>

<div style="text-align: center;">
    <img src="img/Spring_Damper_System.png" width="600">
</div>

A Sample work through is provided for System A, and then provide the necessary steps to solve System B, leaving it as a self-assessment exercise (The solution can be found in **NB01_sol.ipynb**).

Note that we want to create a controller that allows us to follow a reference spring force in A, and to follow a reference mass position in B. This is achieved by the force $F_c(t)$ that we control.


### System A


In system A there is a spring, with spring constant $k$, whose natural state is at $x=0$. The point mass hanging on the spring has mass $m$. A force $F_c$ acts on the point mass. For the measurement we have included a force sensor on the base of the spring.

#### Step 1: Defining the boundaries, inputs and outputs

The:
- *inputs:* The endogenous/control input is $F_c$, since it is the only variable external to the system that can affect the system. There are no exogenous inputs/disturbances to the system.
- *outputs:* This is the force of the spring-- note that in this example, this has been selected for illustrative purposes, other options are available. 
- *state:* $x = \begin{bmatrix} x \\ \dot x \end{bmatrix}$, note that another state has been added here in anticipation of the aforementioned method in Step 3.  

#### Step 2: Set up the differential equations

The first step is to draw a free-body-diagram:

<div style="text-align: center;">
    <img src="img/Free_Body_Diagram_2.png" width="400">
</div>

*Note* 
- The force exerted by the spring when stretched compressed is $F_s=k\Delta x$, where k is the spring constant and $\Delta x$ is the deviation from the equilibrium state.
- Both $m$ and $g$ are parameters, and hence, at least in this case, $F_g$ is not an input. 

In this example, the position of the mass is the relevant storage quantity, thus this is the only quantity to keep track of. If we now use Newtons 2nd Law for linear motion as mentioned above we get the following equation:
$$
\sum F = F_c(t) + mg - k x(t) = m \ddot x(t)
$$
We can rewrite this and get ordinary differential equation (ODE):
$$
\ddot x(t) = \frac{1}{m} (F_c(t) + mg - kx(t))
$$
For the output we have a force measurement that is proportional to x(t)
$$
y(t) = k \cdot x(t)
$$ 

#### Step 3: Write down in standard form

Since the ODE has a second derivative, the system is not yet in standard form. We first define our input as
$$
u(t) = F_c(t)
$$
Next we define the following states (we notationally replace $x(t)$ in the definition of the standard form with $z(t)$ to avoid confusion):
$$
z_1(t) = x(t), \quad z_2(t) = \dot x(t) 
\Rightarrow z(t) = 
\begin{bmatrix}
    z_1(t) &
    z_2(t)
\end{bmatrix}^T
$$
With this we get the following:
$$
\dot z(t) = 
\begin{bmatrix}
    \dot z_1(t) \\
    \dot z_2(t)
\end{bmatrix} =
\begin{bmatrix}
    z_2(t) \\
    \frac{1}{m} (u(t) + mg - kz_1(t))
\end{bmatrix}
$$
And the output is:
$$
y(t) = k \cdot z_1(t)
$$
Thus our system is described by:
$$
\dot z(t) = 
\begin{bmatrix}
    z_2(t) \\
    \frac{1}{m} (u(t) + mg - kz_1(t))
\end{bmatrix}
$$
$$
y(t) = k \cdot z_1(t)
$$

As an exercise you can now fill the blanks for system B.

### System B: Exercise

Note that in addition to System A, there is a damper in System B: (Damper Law: $F=b \dot x$, where $b$ is a constant and $\dot x$ is the velocity of the spring.
This time we measure directly the position of $x$

#### Step 1: Defining the boundaries, inputs and outputs
Define the inputs, outputs, ...

#### Step 2: Setup the differential equations
Create the free body diagram and draw the forces accordingly.
<div style="display: flex; justify-content: center;">
    <div style="width: 200px; height: 100px; border: 2px solid black;"></div>
</div>

Using Newtons 2nd Law for linear motion as mentioned above we get:
$$
\sum F = m \ddot x \\
\Rightarrow \square = \square
$$
We can rewrite this and get:
$$
\ddot x(t) = \square
$$
For the output we have a force measurement that is proportional to x(t)
$$
y(t) = \square
$$ 

#### Step 3: Write down in standard form
Since we have a second derivative our system is not yet in standard form. We first define our input as
$$
u(t) = \square
$$
Next we define the following states (we replace $x(t)$ in the definition of the standard from with $z(t)$ to avoid confusion):
$$
z(t) = 
\square
$$
With this we get the following:
$$
\dot z(t) = 
\square
$$
And the output is:
$$
y(t) = \square
$$


# Computing the Output Signal


Once we have a system in standard form we would then like to compute the output of that system for different inputs -- e.g., consider the cruise control example, we need to know how the speed of the vehicle evolves over time.

However calculating the output in a closed form solution ($x(t) = g(x_0, u(t))$) is sometimes analytically not possible. <br>
Nevertheless, we know that over some time $\Delta t$ the system evolves as follows:
$$
x(t+\Delta t) = \int_{t}^{t+\Delta t} f(x(\tau), u(\tau)) d\tau
$$
This calculation is also not trivial but for sufficiently small $\Delta t$ we can approximate this integral using:
$$
x(t+\Delta t) \approx x(t) + \Delta t \cdot f(x(t), u(t))
$$
*Note: An alternative interpretation of this formula comes from the approximation of the derivative (rearranging this formula results in the same as above):*
$$
\dot x = f(x(t), u(t)) \approx \frac{x(t+\Delta t)- x(t)}{\Delta t}
$$

Thus, using the above it is possible to calculate the output for a given system.
Later in the course, you will see other (analytical) approaches to compute the output response (such as in Lecture 4). 

To help develop an intuitive understanding of how varying the time steps affects the output, we have provided an interactive example below, that calculates the trajectory of the spring damper system A.

Load the simulator:

In [6]:
class simulator:
    
    def __init__(self):
        # define the parameters of your system
        self.m = 1                  #kg
        self.g = 9.81               #m/s^2
        self.t_end = 15             #s

    def nonlinear_dynamics(self, x, u, t, damper_on, k, b):
        # define the nonlinear dynamics f(x(t), u(t)) of the system
        return np.array([x[1], self.g - k/self.m*x[0] + u/self.m - b/self.m*x[1]*damper_on])
    
    def euler(self, x_0, k, b, damper_on, delta_t):
        # simple euler integration simulator
        x = np.copy(x_0)
        x_history = np.copy(x_0)
        t = 0
        N = int(self.t_end / delta_t)
        c = 0
        while c < N:
            u = 0
            x = x + delta_t * self.nonlinear_dynamics(x, u, t, damper_on, k, b)
            t += delta_t
            c += 1
            x_history = np.vstack((x_history, x))
        return x_history
    
model = simulator()
def run_sim(k, b, damper_on=0, dt=1):
    x_0 = np.array([0, 0])
    x_history = model.euler(x_0, k, b, damper_on, dt/1000)
    return x_history
    

### Interactive Exercise


In the example you can vary the spring constant and damping constant and see how these affect the system.
Additionally, a slider which indicates the step size $\Delta t$ (in ms). By varying the time step, you should see that using such an approach to calculate the output, can sometimes return inaccurate solutions.
- When the step size is large the output response is not faithful to the actual system response due to discretization errors; 
- When the step size is small, the output is faithful to the actual system response, but the simulation takes longer to run, potentially not terminating in finite time. 

In some applications or examples (such as the one below), it is possible to determine a time step such that the output response is faithful to that of the actual system -- but keep in mind that this is not always true by construction.

In [7]:
## Code for interactive plot
# Initialize a display object
output = widgets.Output()
# Function to update the plot and title
def update_plot(k_slider, b_slider, dt_slider):
    with output:
        output.clear_output(wait=True)  # Clear the previous output
        # two subplots next to each other with given figure size:
        fig, axs = plt.subplots(1, 2, figsize=(20,5))
        x_out = run_sim(k=k_slider, b=b_slider, damper_on=0, dt=dt_slider)
        axs[0].plot(x_out[:,0], label='No Damper', color='red')
        axs[1].plot(x_out[:,1], label='No Damper', color='red')
        x_out = run_sim(k=k_slider, b=b_slider, damper_on=1, dt=dt_slider)
        axs[0].plot(x_out[:,0], label='Damper', color='orange')
        axs[1].plot(x_out[:,1], label='Damper', color='orange')

        # x lable with latex format of theta
        axs[0].set_xlabel(r'Time: t[s]')
        axs[0].set_ylabel(r'Position: $x(t) [m]$')
        axs[1].set_xlabel(r'Time: t[s]')
        axs[1].set_ylabel(r'Velocity: $\dot x(t)[m/s]$')
        axs[0].set_title('Time response (dt = '+str(dt_slider/1000)+'[s]'+")")
        
        for ax in axs:
            xticks = ax.get_xticks()  # Get current x-ticks
            ax.set_xticks(xticks)  # Fix the ticks first
            ax.set_xticklabels([f'{int(tick*dt_slider/1000)}' for tick in xticks])  # Divide x-ticks by 100

        axs[0].legend()
        axs[1].legend()
        axs[0].grid()
        axs[1].grid()
        plt.show()

k_slider = widgets.FloatSlider(value=1, min=0, max=10, step=0.1, description="Spring constant")
b_slider = widgets.FloatSlider(value=1, min=0, max=5, step=0.1, description="Damping coefficient")
dt_slider = widgets.FloatSlider(value=10, min=1, max=500, step=0.1, description="Time step [ms]")

# Arrange the sliders vertically
controls_box = widgets.VBox([k_slider, b_slider, dt_slider])

# Link the widgets to the update_plot function
widgets.interactive_output(update_plot, {
    'k_slider': k_slider,
    'b_slider': b_slider,
    'dt_slider': dt_slider
})

# Display the controls and the plot
display(controls_box, output)


Output()

# Benefits/Dangers of Feedback

In general there are several benefits and drawbacks to feedback:
<p style="margin-left: 20px">+ Can stabilize an otherwise unstable system.</p>
<p style="margin-left: 20px">+ Handle uncertainties in the system.</p>
<p style="margin-left: 20px">+ Handle disturbances.</p>
<p style="margin-left: 20px">- May introduce instability in an otherwise stable system.</p>
<p style="margin-left: 20px">- May feed sensor noise into the system.</p>

To help concretize the above, below we provide an interactive example where the 1<sup>st</sup>, 3<sup>rd</sup>, and 4<sup>th</sup> points are illustrated. Consider the mechanical system depicted below.

> The system consists of a rigid bar of length $L$ and mass $m$ which is mounted with a fixed bearing at its lower end. The bearing has friction and causes a frictional moment $\tau_f(t)=c_f\dot\theta(t)$, where $c_f$ and $\dot\theta(t)$ represent the friction coefficient and the rod's angular velocity respectively. 
Assume that there is a motor that lets you regulate a torque $\tau(t)$ which is applied to the rod. The moment of inertia of the bar with respect to the point $P$ is given by $J = \frac{1}{3}mL^2$. The angular position $\theta(t)$ of the rod is measured with a sensor.
The objective is to control the pendulum's position $\theta(t)$ and to keep it in the upright position $\theta(t)=0$, $\forall t$.

<div style="text-align: center;">
    <img src="img/Inverted_Pendulum.png" width="200">
</div>

The first step would be to model the system. This is skipped here and one can perform it as part Exercise 4 in PS01.

When running the code one can see how the system naturally behaves. It starts upright until at $t=3$ when an impulse torque notches the pendulum out of its unstable equilibrium. The pendulum then swings down and approaches its stable equilibrium -- at 180° ($\pi$ rad). This is what we would expect from this system.

Now the goal is to introduce a feedback controller to stabilize the system to an upright position. See below for the feedback architecture. 

<div style="text-align: center;">
    <img src="img/Control_Loop.png" width="800">
</div>

### Interactive Example

Below we introduce a bit more formally the type of controller implemented for the example. Note that this will be covered later on the course -- so feel free to revisit it later on, but take note of the possible behaviors that feedback may induce.

Suppose we want the system to be at the desired angle $\theta_{ref} = 0$.
We can then define the error of the system as the deviation of the angle from the reference angle:
$$
e(t) = \theta_{ref}-\theta(t)
$$
Then the controller should input a control signal proportional to the error:
$$
u_p(t) = K_p \cdot e(t)
$$.

To see what happens you can turn on the controller and try different values for $K_p$ using the slider (leave $K_d=0$):
1. One can see that for values $K_p < 4.80$ the control is not able to reach the desired state of $\theta = 0$. This is since not enough control torque is applied to the system.
2. When increasing the gain above $4.80$ (until a certain threshold) the system reaches the desired state, thus stabilising the system. When increasing the $K_p$ gain the system approaches the desired value faster. However we see that the system overshoots and also oscillates a bit around the desired angle, before approaching the desired angle.
3. If increasing $K_p$ *too much*, then the desired value is not reached, leading to an unstable system. Demonstrating that one has to be careful when implementing feedback as it can create a controller that will damage or destroy your system.

Thus, it is important to note that by construction, simply implementing feedback does not mean that the system is always stable. Nevertheless, for the provided example, it is possible to ensure that the system is always stable (i.e., reduce the overshooting and oscillations) by considering a different controller (a PD Controller). 

For this we calculate the rate of change of the error (i.e. are we make a lot of progress towards the right direction or not). We define:
$$
\dot e(t) = \frac{d}{dt}(\theta_{ref}- \theta(t)) \underbrace{=}_{\text{since } \dot \theta_{ref}\text{ is zero}} - \dot \theta(t)
$$
We now introduce this term into the control input as follows:
$$
u_{PD}(t) = K_p \cdot e(t) + K_d \cdot \dot e(t)
$$



Set $K_p = 20$ and then try different values for $K_d$ using the slider:
1. It can be seen that increasing $K_d$ value lowers the oscillation, where it always reached the desired $\dot \theta_{ref}$. One can interpret this as a sort of damping, when we are moving fast towards the desired goal the controller backs of to "gently" approach the desired goal.

Feel free to play around with different values of $K_p$ and $K_d$ and see how the system behaves.

In [9]:
class simulator:
    
    def __init__(self):
        # define the parameters of your system
        self.m = 1                  #kg
        self.g = 9.81               #m/s^2
        self.l = 1                  #m
        self.cf = 0.5               #Nms/rad
        self.impulse_torque = 10    #Nm
        self.impulse_duration = 0.1 #s
        self.impulse_time = 3       #s
        self.t_end = 20             #s
        self.delta_t = 0.01         #s

    def nonlinear_dynamics(self, x, u, t):
        # define the nonlinear dynamics f(x(t), u(t)) of the system
        disturbance = 0
        if t > self.impulse_time and t < self.impulse_time + self.impulse_duration:
            disturbance = self.impulse_torque
        return np.array([x[1],
                        3*self.g/(2*self.l)*np.sin(x[0]) 
                        - 3*self.cf/(self.m*self.l**2)*x[1] 
                        + 3/(self.m*self.l**2)*u
                        + 3/(self.m*self.l**2)*disturbance])
    
    def euler(self, control_on, x_0, Kp, Kd):
        # euler integration simulator
        x = np.copy(x_0)
        x_history = np.copy(x_0)
        t = 0
        while t < self.t_end:
            u = self.control(x, Kp, Kd) if control_on else 0
            x = x + self.delta_t * self.nonlinear_dynamics(x, u, t)
            t += self.delta_t
            x_history = np.vstack((x_history, x))
        return x_history
    
    def control(self, x, Kp, Kd):
        # simple PD controller
        u = - Kp*x[0] - Kd*x[1]
        return u
    
model = simulator()
def run_sim(control_on, K_p, K_d):
    x_0 = np.array([0, 0])
    x_history = model.euler(control_on, x_0, K_p, K_d)
    return x_history
    

In [10]:
## Code for interactive plot
# Initialize a display object
output = widgets.Output()

# Function to update the plot and title
def update_plot(control_off_checkbox, control_on_checkbox, Kp_slider, Kd_slider):
    with output:
        output.clear_output(wait=True)  # Clear the previous output
        # two subplots next to each other with given figure size:
        fig, axs = plt.subplots(1, 2, figsize=(20,5))
        
        if control_off_checkbox:
            x_out = run_sim(control_on=False, K_p=Kp_slider, K_d=Kd_slider)
            axs[0].plot(x_out[:,0], label='No Feedback', color='red')
            axs[1].plot(x_out[:,1], label='No Feedback', color='red')
        
        if control_on_checkbox:
            x_out = run_sim(control_on=True, K_p=Kp_slider, K_d=Kd_slider)
            axs[0].plot(x_out[:,0], label='With Feedback', color='blue')
            axs[1].plot(x_out[:,1], label='With Feedback', color='blue')
        
        # x label with latex format of theta
        axs[0].set_xlabel(r'Time: t[s]')
        axs[0].set_ylabel(r'Angle: $\theta(t) [rad]$')
        axs[1].set_xlabel(r'Time: t[s]')
        axs[1].set_ylabel(r'Angular Velocity: $\dot\theta(t)[rad/s]$')
        for ax in axs:
            xticks = ax.get_xticks()  # Get current x-ticks
            ax.set_xticks(xticks)  # Fix the ticks first
            ax.set_xticklabels([f'{int(tick/100)}' for tick in xticks])  # Divide x-ticks by 100

        if control_off_checkbox or control_on_checkbox: # avoids legend error if none of the functions are selected
            axs[0].legend()
            axs[1].legend()
        axs[0].grid()
        axs[1].grid()
        plt.show()

# Create checkboxes to show controlled and uncontrolled plots
control_off_checkbox = widgets.Checkbox(value=True, description='Control off')
control_on_checkbox = widgets.Checkbox(value=False, description='Control on')

# Create a slider for Kp and Kd
Kp_slider = widgets.FloatSlider(value=0, min=0, max=70, step=0.1, description="K_p gain")
Kd_slider = widgets.FloatSlider(value=0, min=0, max=5, step=0.1, description="K_d gain")

# Arrange checkboxes in a vertical box
checkboxes_box = widgets.VBox([control_off_checkbox, control_on_checkbox])

# Arrange the checkboxes box and slider in a horizontal box
controls_box = widgets.HBox([checkboxes_box, widgets.VBox([Kp_slider, Kd_slider])])

# Link the widgets to the update_plot function
widgets.interactive_output(update_plot, {
    'control_off_checkbox': control_off_checkbox,
    'control_on_checkbox': control_on_checkbox,
    'Kp_slider': Kp_slider,
    'Kd_slider': Kd_slider
})

# Display the controls and the plot
display(controls_box, output) 

Output()